


# Escenarios experimentales Internetworking
Presentado por: Jonathan Toapanta
Fecha: 23/02/2023



In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #Gives us Graphics
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [28]:
# cargar datos y normalizar
# Load and preprocess the data
url = 'https://raw.githubusercontent.com/beespinosa1/Inter/main/Escenario2.csv'
df = pd.read_csv(url)
df = pd.read_csv(url,low_memory=False)
dataframe = pd.read_csv(url,low_memory=False)


In [29]:
def dfNormalize(df):
    for feature_name in df.columns:
        df.loc[:,feature_name] = pd.to_numeric(df.loc[:,feature_name], errors='coerce').fillna(0)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()   
        if (max_value - min_value) > 0:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value) / (max_value - min_value)
        else:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value)    
    return df

In [30]:
dataframe = dataframe.reindex(np.random.permutation(dataframe.index)).copy()

keys = dataframe.keys()
data_to_process = dataframe[keys[4:len(keys)-1]].copy()
x_normalised = dfNormalize(data_to_process)


In [31]:
y = dataframe['label']
change_labels = lambda x: 1 if x == 'nonTOR' else 0
y_normalised = dataframe['label'].apply(change_labels)



In [32]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_normalised, y_normalised, test_size=0.2, random_state=42)

# Reshape the data to fit the input shape of the CNN
x_train = x_train.values.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.values.reshape(x_test.shape[0], x_test.shape[1], 1)


In [33]:
# Convert the labels to categorical format
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [34]:
# Define the Deep Transfer Learning model
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=x_train.shape[1:]))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))

In [35]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [36]:
# Train the model
history = model.fit(x_train, y_train, batch_size=32, epochs=20, validation_data=(x_test, y_test))



Epoch 1/20
75/75 [==============================] - 2s 10ms/step - loss: 0.1182 - accuracy: 0.9871 - val_loss: 0.0597 - val_accuracy: 0.9983
Epoch 2/20
75/75 [==============================] - 1s 8ms/step - loss: 0.0132 - accuracy: 0.9992 - val_loss: 0.0207 - val_accuracy: 0.9983
Epoch 3/20
75/75 [==============================] - 1s 8ms/step - loss: 0.0073 - accuracy: 0.9992 - val_loss: 0.0236 - val_accuracy: 0.9983
Epoch 4/20
75/75 [==============================] - 1s 8ms/step - loss: 0.0078 - accuracy: 0.9992 - val_loss: 0.0190 - val_accuracy: 0.9983
Epoch 5/20
75/75 [==============================] - 1s 8ms/step - loss: 0.0079 - accuracy: 0.9992 - val_loss: 0.0231 - val_accuracy: 0.9983
Epoch 6/20
75/75 [==============================] - 1s 8ms/step - loss: 0.0071 - accuracy: 0.9992 - val_loss: 0.0254 - val_accuracy: 0.9983
Epoch 7/20
75/75 [==============================] - 1s 10ms/step - loss: 0.0076 - accuracy: 0.9992 - val_loss: 0.0257 - val_accuracy: 0.9983
Epoch 8/20
75/75 [

In [37]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_bin = (y_pred > 0.5).astype(int)
    loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
    precision = precision_score(y_test, y_pred_bin, average='weighted')
    recall = recall_score(y_test, y_pred_bin, average='weighted')
    f1 = f1_score(y_test, y_pred_bin, average='weighted')
    cm = confusion_matrix(y_test.argmax(axis=1), y_pred_bin.argmax(axis=1))
    print("Confusion Matrix:\n", cm)
    print("Loss: {:.4f}\nAccuracy: {:.4f}\nPrecision: {:.4f}\nRecall: {:.4f}\nF1 Score: {:.4f}".format(loss, accuracy, precision, recall, f1))


In [38]:
evaluate_model(model, x_test, y_test)

19/19 [==============================] - 0s 2ms/step
Confusion Matrix:
 [[  0   1]
 [  0 599]]
Loss: 0.0281
Accuracy: 0.9983
Precision: 0.9967
Recall: 0.9983
F1 Score: 0.9975


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
